In [1]:
%reload_ext autoreload
%autoreload 2
import os,sys
sys.path.insert(1, os.path.join(sys.path[0],
                                '..', 'module'))
import wiki
import pickle
import numpy as np
import gensim.models as gm
import gensim.utils as gu
import gensim.corpora as gc
import gensim.matutils as gmat
import matplotlib.pyplot as plt
import sklearn.metrics.pairwise as smp

### Test big model
See `preparation/build-tfidf-model.ipynb`.

In [5]:
path_save = '/Users/harangju/Developer/data/wiki/models/'
tfidf = gu.SaveLoad.load(path_save + 'tfidf.model')
dct = pickle.load(open(path_save + 'dict.model','rb'))

In [6]:
corpus[1][:5]

['transhumanism', 'abbreviated', 'as', 'or', 'is']

In [7]:
tfidf[dct.doc2bow(corpus[1])][:5]

[(4, 0.0014903867352161592),
 (5, 0.22276491051599304),
 (9, 0.0016905889256660094),
 (12, 0.04580387652629316),
 (13, 0.2198398784494291)]

### Weight graph

In [ ]:
topic = 'earth science'
network = wiki.Net()
network.load_graph('/Users/harangju/Developer/data/wiki/graphs/'+topic+'.gexf')

In [ ]:
def set_weights(graph, dump, model, dct):
    for n1, n2 in graph.edges:
        p1 = gu.simple_preprocess(dump.load_page(n1).strip_code())
        p2 = gu.simple_preprocess(dump.load_page(n2).strip_code())
        graph[n1][n2]['weight'] = smp.cosine_similarity(X=model[dct.doc2bow(p1)],
                                                        Y=model[dct.doc2bow(p2)])

In [ ]:
network.graph['Biosphere']

In [ ]:
print(network.graph['Biosphere']['Earth'])
print(network.graph['Biosphere']['Earth'])

In [ ]:
set_weights(network.graph, dump, model, dct)

In [ ]:
print(network.graph['Biosphere']['Earth'])
print(network.graph['Biosphere']['Earth'])